# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [240]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import math
import numpy as np


In [241]:
df_1 = pd.read_csv('/datasets/geo_data_0.csv')
display(df_1.info())
df_1.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [242]:
df_2 = pd.read_csv('/datasets/geo_data_1.csv')
display(df_2.info())
df_2.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [243]:
df_3 = pd.read_csv('/datasets/geo_data_2.csv')
display(df_2.info())
df_3.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


##### Вывод

В данных пропусков нет и записаны в верном типе данных

## Обучение и проверка модели

In [244]:
df = [df_1, df_2, df_3]

In [245]:
predicted_valid_list = []
predicted_valid_mean = []
target_valid_list = []
def rmse(df):
    count = 1
    for i in df:
        target = i['product']
        features = i.drop(['product', 'id'], axis=1)
        features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
        model = LinearRegression()
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        predicted_valid_list.append(predicted_valid)
        predicted_valid_mean.append(predicted_valid.mean())
        target_valid_list.append(target_valid)
        mse = mean_squared_error(target_valid, predicted_valid)
        print(f'Средний запас предсказанного сырья в {count} регионе = {predicted_valid.mean()}')
        print("RMSE =", mse ** 0.5)
        count += 1

        
rmse(df)

Средний запас предсказанного сырья в 1 регионе = 92.59256778438035
RMSE = 37.5794217150813
Средний запас предсказанного сырья в 2 регионе = 68.728546895446
RMSE = 0.893099286775617
Средний запас предсказанного сырья в 3 регионе = 94.96504596800489
RMSE = 40.02970873393434


In [246]:
predicted_valid_list

[array([ 95.89495185,  77.57258261,  77.89263965, ...,  61.50983303,
        118.18039721, 118.16939229]),
 array([ 82.66331365,  54.43178616,  29.74875995, ..., 137.87934053,
         83.76196568,  53.95846638]),
 array([ 93.59963303,  75.10515854,  90.06680936, ...,  99.40728116,
         77.77991248, 129.03241718])]

In [247]:
predicted_valid_mean

[92.59256778438035, 68.728546895446, 94.96504596800489]

In [248]:
target_valid_list

[71751     10.038645
 80493    114.551489
 2655     132.603635
 53233    169.072125
 91141    122.325180
             ...    
 12581    170.116726
 18456     93.632175
 73035    127.352259
 63834     99.782700
 43558    177.821022
 Name: product, Length: 25000, dtype: float64,
 71751     80.859783
 80493     53.906522
 2655      30.132364
 53233     53.906522
 91141      0.000000
             ...    
 12581    137.945408
 18456    110.992147
 73035    137.945408
 63834     84.038886
 43558     53.906522
 Name: product, Length: 25000, dtype: float64,
 71751     61.212375
 80493     41.850118
 2655      57.776581
 53233    100.053761
 91141    109.897122
             ...    
 12581     28.492402
 18456     21.431303
 73035    125.487229
 63834     99.422903
 43558    127.445075
 Name: product, Length: 25000, dtype: float64]

##### Вывод

Средние запасы в первом и третьем регионе самые большие.
Средняя ошибка во втором регионе минимальная.

## Подготовка к расчёту прибыли

In [249]:
budget = 10000000000  # бюджет на разработку 200 скважин
best_wells = 200 # количество скважин
unit_income = 450000 # доход с каждой единицы продукта
min_profit = budget / best_wells # минимальная прибыль одной скважины
print('Минимальная прибыль одной скважины', min_profit, 'руб.')
minimum_volume = math.ceil(min_profit / unit_income)
print('Минимальная объём сырья для безубыточной разработки новой скважины', minimum_volume, 'тыс. баррелей')

Минимальная прибыль одной скважины 50000000.0 руб.
Минимальная объём сырья для безубыточной разработки новой скважины 112 тыс. баррелей


#### Вывод

Средние запасы нефти по трем регионам меньше, чем требуется для безубыточной разработки скважин

## Расчёт прибыли и рисков 

In [250]:
# функция расчета прибыли
def revenue(target, predicted, count):
    predict_sorted = pd.Series(predicted).sort_values(ascending=False)
    volume = target.reset_index(drop=True)[predict_sorted.index][:count]
    return unit_income * volume.sum() - budget

In [251]:
print(revenue(target_valid_list[0], predicted_valid_list[0], 200))
print(revenue(target_valid_list[1], predicted_valid_list[1], 200))
print(revenue(target_valid_list[2], predicted_valid_list[2], 200))

3320826043.1398506
2415086696.681511
2710349963.5998325


In [252]:
count = 0

for index in range(len(target_valid_list)):    
    target = target_valid_list[index].reset_index(drop=True)
    probabilities = predicted_valid_list[index]

    state = np.random.RandomState(12345)

    values = []
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        probs_subsample = probabilities[target_subsample.index] 
        result = revenue(target_subsample, probs_subsample, 200)
        values.append(result)
    values = pd.Series(values)

    lower = values.quantile(0.025) / 1000000
    upper = values.quantile(0.975) / 1000000
    profit = values.mean()
    negative = (values[values<0].count() / len(values)) * 100
    count += 1
    print(f'{count} регион')
    print(f'\tС вероятностью 95% прибыль составит от {lower:,.2f} до {upper:,.2f} млн. руб.')
    print(f'\tСредняя прибыль равняется {profit/ 1000000:,.2f} млн. руб. ')
    print(f'\tРиск убытков состовляет {negative}%\n')

1 регион
	С вероятностью 95% прибыль составит от -111.22 до 909.77 млн. руб.
	Средняя прибыль равняется 396.16 млн. руб. 
	Риск убытков состовляет 6.9%

2 регион
	С вероятностью 95% прибыль составит от 33.82 до 852.29 млн. руб.
	Средняя прибыль равняется 456.05 млн. руб. 
	Риск убытков состовляет 1.5%

3 регион
	С вероятностью 95% прибыль составит от -163.35 до 950.36 млн. руб.
	Средняя прибыль равняется 404.40 млн. руб. 
	Риск убытков состовляет 7.6%



#### Вывод 

Написал функцию расчета прибыли и применил технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

Лучшие показатели показывает второй регион.

## Общий вывод

Провел исследования в трех регионах. В исследовании использовал модель линейной регрессии. Сохранил все предсказания в списке. 

Рассчитал достаточный объем сырья для безубыточной разработки новой скважины, он равен 112 тыс. баррелей. Средние запасы в каждом регионе ниже минимального объема сырья для безубыточной разработки. 

Выбрал 200 скважин с максимальными объемами и по ним провел расчет прибыли и применил технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли. Наилучшим оказался второй регион со средней прибылью 456.05 млн. руб, риск убытка составляет 1,5%.